**Run the following two cells before you begin.**

In [34]:
%autosave 10

Autosaving every 10 seconds


In [35]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [36]:
# Import the data set
df = pd.read_csv('Cleaned_Data.csv')

In [76]:
# Define the sigmoid function
def sigmoid(Z):
    Y = 1 / (1 + np.exp(-Z))
    return Y

In [38]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [77]:
# Create a train/test split
X = df[['PAY_1','LIMIT_BAL']].values
y = df['default payment next month'].values
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 24)




______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [78]:
from sklearn.linear_model import LogisticRegression
my_lr = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [79]:
# Fit the logistic regression model on training data
my_lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [80]:
# Make predictions using `.predict()`
pred = my_lr.predict(X_test)

In [81]:
# Find class probabilities using `.predict_proba()`
class_prob = my_lr.predict_proba(X_test)
class_prob

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [82]:
# Add column of 1s to features
ones_and_features = np.hstack([np.ones((X_test.shape[0],1)), X_test])
print(ones_and_features)
np.ones((X_test.shape[0],1)).shape

[[ 1.0e+00  2.0e+00  1.6e+05]
 [ 1.0e+00  1.0e+00  5.0e+04]
 [ 1.0e+00 -1.0e+00  2.0e+05]
 ...
 [ 1.0e+00 -1.0e+00  5.0e+04]
 [ 1.0e+00  1.0e+00  2.3e+05]
 [ 1.0e+00  2.0e+00  1.0e+05]]


(5333, 1)

In [83]:
# Get coefficients and intercepts from trained model
coef = lr_model.coef_
intercept = lr_model.intercept_.reshape(1,1)
intercept_and_coefs = np.concatenate([intercept, coef], axis=1)
intercept_and_coefs

array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [84]:
# Manually calculate predicted probabilities

X_lin_comb = np.dot(intercept_and_coefs, np.transpose(ones_and_features))
y_pred_proba_manual = sigmoid(X_lin_comb)
y_pred_proba_manual

array([[0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
        0.33606565]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [85]:
# Manually calculate predicy_pred_manual = y_pred_proba_manual >= 0.5
predicy_pred_manual = y_pred_proba_manual >= 0.5
predicy_pred_manual.shape



(1, 5333)

In [86]:
# Compare to scikit-learn's predicted classes
np.array_equal(class_prob.reshape(1,-1), predicy_pred_manual)


False

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [87]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_proba_manual.reshape(y_pred_proba_manual.shape[1],))


0.627207450280691

In [90]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, class_prob[:,1])

0.627207450280691